In [97]:
import pickle
import numpy as np
import pandas as pd

DATA = {
    "X" : [],
    "category" : [],
    "y" : []
}

X=DATA["X"]
category=DATA["category"]
y=DATA["y"]


with open('covid_fake_news_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(0,len(dst)))
    data = pd.read_csv("raw/covid_fake_news.csv",sep="\t")
    y.extend(data["label"].values)
    
with open('mmcovid_en_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(1,len(dst)))
    data = pd.read_csv("raw/mmcovid_en.csv",sep=",")
    y.extend(data["label"].values)
    
# with open('isot_BERTEmbeddings.pickle', 'rb') as handle:
#     dst = pickle.load(handle)
#     X.extend(dst)
#     category.extend(np.repeat(3,len(dst)))
    
# with open('grafn_BERTEmbeddings.pickle', 'rb') as handle:
#     dst = pickle.load(handle)
#     X.extend(dst)
#     category.extend(np.repeat(4,len(dst)))
    
with open('pubhealth_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(2,len(dst)))
    data = pd.read_csv("raw/pubhealth.csv",sep=",")
    y.extend(data["label"].values)

    
# with open('qprop_BERTEmbeddings.pickle', 'rb') as handle:
#     dst = pickle.load(handle)
#     print(dst.shape)
#     X.extend(dst)
#     category.extend(np.repeat(3,len(dst)))
#     data = pd.read_csv("raw/qprop.csv",sep="\t")
#     y.extend(data["label"].values)
#     print(data.shape)
    
DATA["X"]=np.array(X)
DATA["category"]=np.array(category)
DATA["y"]=np.array(y)
DATA["folds"] = []

In [96]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def experiment(foldids, X, y, cls = LogisticRegression(max_iter=10000), fit=True):

    scores = {
        'Accuracy': {'func': accuracy_score},
        'Balanced Accuracy': {'func': balanced_accuracy_score},
        'F1': {'func': f1_score},
        'Precision': {'func': precision_score},
        'Recall': {'func': recall_score},
        'G-mean': {'func': geometric_mean_score}
    }

    for score_name, score_dict in scores.items():
        scores[score_name]["list"] = []
        scores[score_name]["lab"] = []

    for fold,j in enumerate(foldids):
        train = foldids[fold][1]
        test = foldids[fold][2]
        xin, yin = X[train], np.array(y[train])
        if fit == True:
            cls.fit(xin, yin)
        y_pred = cls.predict(X[test])
        for score_name, score_dict in scores.items():
            if score_name in ["F1","Precision","Recall"]:
                scorvaln = score_dict['func'](y[test], y_pred, average=None)
                score_dict['lab'].append(scorvaln)
                scorval = score_dict['func'](y[test], y_pred, average="weighted")
                score_dict['list'].append(scorval)
                print(score_name, scorval, scorvaln)  
            else:
                scorval=score_dict['func'](y[test], y_pred)
                score_dict['list'].append(scorval)
                print(score_name, scorval)
        print(" ")

    clear_output()
    for score_name, score_dict in scores.items():
        score_dict['avg'] = np.mean(score_dict['list'])
        score_dict['std'] = np.std(score_dict['list'])
 
    # Print stats
    numlabels = scores["F1"]["lab"][0].shape[0]
    scores["F1"]["lab"][0].shape[0] 
    head = "| %-20s | %-10s |" +  numlabels * " %-10s |" 
    headv = ["Score", "Average"]
    headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
    row=head % tuple(headv)
    print("+"*len(row))
    print(row)
    print("+"*len(row))
    for score_name, score_dict in sorted(scores.items()) :
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
        for i in range(numlabels):
            if score_name in ["F1","Precision","Recall"]:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels* " %4.1f ± %3.1f |"
                vals = [v[i] for v in scores[score_name]["lab"]]
                headv.append(np.mean(vals)*100)
                headv.append(np.std(vals)*100)
            else:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels * " %-10s |" 
                headv.append("-")
        print(head % tuple(headv))
    print("+"*len(row))
    return cls, scores

<hr style="border: 5px dashed red">

In [98]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.decomposition import PCA




X = DATA["X"][DATA["category"] == 0]
y = DATA["y"][DATA["category"] == 0]

pca = PCA(n_components=85)
pca.fit_transform(X)


rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

DATA["folds"].append(foldids)
model1, scores1 = experiment(foldids, pca.transform(X), y, LogisticRegression(max_iter=10000))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 96.8 ± 0.2 | -          | -          |
| Balanced Accuracy    | 76.6 ± 1.2 | -          | -          |
| F1                   | 96.6 ± 0.2 | 98.3 ± 0.1 | 63.6 ± 1.8 |
| G-mean               | 73.2 ± 1.7 | -          | -          |
| Precision            | 96.5 ± 0.2 | 97.6 ± 0.1 | 77.3 ± 3.2 |
| Recall               | 96.8 ± 0.2 | 99.1 ± 0.2 | 54.1 ± 2.5 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [99]:
X = DATA["X"][DATA["category"] == 1]
y = DATA["y"][DATA["category"] == 1]

pca = PCA(n_components=85)
pca.fit_transform(X)

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

DATA["folds"].append(foldids)
model2, scores2 = experiment(foldids, pca.transform(X), y, LogisticRegression(max_iter=10000))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 91.7 ± 0.3 | -          | -          |
| Balanced Accuracy    | 86.9 ± 0.7 | -          | -          |
| F1                   | 91.4 ± 0.3 | 94.4 ± 0.2 | 83.5 ± 0.7 |
| G-mean               | 86.3 ± 0.8 | -          | -          |
| Precision            | 91.7 ± 0.2 | 91.5 ± 0.5 | 92.3 ± 1.1 |
| Recall               | 91.7 ± 0.3 | 97.5 ± 0.4 | 76.3 ± 1.8 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [100]:
X = DATA["X"][DATA["category"] == 2]
y = DATA["y"][DATA["category"] == 2]

pca = PCA(n_components=85)
pca.fit_transform(X)

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

DATA["folds"].append(foldids)
model2, scores2 = experiment(foldids, pca.transform(X), y, LogisticRegression(max_iter=10000))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 79.5 ± 0.4 | -          | -          |
| Balanced Accuracy    | 77.9 ± 0.3 | -          | -          |
| F1                   | 79.5 ± 0.4 | 72.4 ± 0.4 | 83.7 ± 0.3 |
| G-mean               | 77.7 ± 0.3 | -          | -          |
| Precision            | 79.4 ± 0.3 | 72.9 ± 0.7 | 83.3 ± 0.3 |
| Recall               | 79.5 ± 0.4 | 71.9 ± 0.6 | 84.0 ± 0.6 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [101]:
X = DATA["X"]
y = DATA["y"]

pca = PCA(n_components=85)
pca.fit_transform(X)

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

model3, scores3 = experiment(foldids, pca.transform(X), y, LogisticRegression(max_iter=10000))

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Score                | Average    | Kat_1      | Kat_2      |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
| Accuracy             | 83.4 ± 0.2 | -          | -          |
| Balanced Accuracy    | 80.2 ± 0.3 | -          | -          |
| F1                   | 83.2 ± 0.2 | 87.8 ± 0.1 | 74.0 ± 0.3 |
| G-mean               | 79.6 ± 0.3 | -          | -          |
| Precision            | 83.2 ± 0.2 | 85.9 ± 0.2 | 77.7 ± 0.3 |
| Recall               | 83.4 ± 0.2 | 89.8 ± 0.2 | 70.6 ± 0.6 |
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [7]:
""" Alpha version of a version of ELLA that plays nicely with sklearn
	@author: Paul Ruvolo
"""

from math import log
import numpy as np
from scipy.special import logsumexp
from scipy.linalg import sqrtm, inv, norm
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression, Lasso
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, explained_variance_score


class ELLA(object):
    """ The ELLA model """
    def __init__(self, d, k, base_learner, base_learner_kwargs = {}, mu = 1, lam = 1, k_init = False):
        """ Initializes a new model for the given base_learner.
            d: the number of parameters for the base learner
            k: the number of latent model components
            base_learner: the base learner to use (currently can only be
                LinearRegression, Ridge, or LogisticRegression).
            base_learner_kwargs: keyword arguments to base learner (for instance to
                                 specify regularization strength)
            mu: hyperparameter for sparsity
            lam: L2 penalty on L
            mu: the L_1 penalty to use
            lam: the L_2 penalty to use
            NOTE: currently only binary logistic regression is supported
        """
        self.d = d
        self.k = k
        self.L = np.random.randn(d,k)
        self.A = np.zeros((d * k, d * k))
        self.b = np.zeros((d * k, 1))
        self.S = np.zeros((k, 0))
        self.T = 0
        self.mu = mu
        self.lam = lam
        self.k_init = k_init
        if base_learner in [LinearRegression, Ridge]:
            self.perf_metric = explained_variance_score
        elif base_learner in [LogisticRegression]:
            self.perf_metric = accuracy_score
        else:
            raise Exception("Unsupported Base Learner")

        self.base_learner = base_learner
        self.base_learner_kwargs = base_learner_kwargs

    def fit(self, X, y, task_id):
        """ Fit the model to a new batch of training data.  The task_id must
            start at 0 and increase by one each time this function is called.
            Currently you cannot add new data to old tasks.

            X: the training data
            y: the trianing labels
            task_id: the id of the task
        """
        self.T += 1
        single_task_model = self.base_learner(fit_intercept = False, **self.base_learner_kwargs).fit(X, y)
        D_t = self.get_hessian(single_task_model, X, y)
        D_t_sqrt = sqrtm(D_t)
        theta_t = single_task_model.coef_

        sparse_encode = Lasso(alpha = self.mu / (X.shape[0] * 2.0),
                              fit_intercept = False).fit(D_t_sqrt.dot(self.L),
                                                         D_t_sqrt.dot(theta_t.T))
        if self.k_init and task_id < self.k:
            sparse_coeffs = np.zeros((self.k,))
            sparse_coeffs[task_id] = 1.0
        else:
            sparse_coeffs = sparse_encode.coef_
        self.S = np.hstack((self.S, np.matrix(sparse_coeffs).T))
        self.A += np.kron(self.S[:,task_id].dot(self.S[:,task_id].T), D_t)
        self.b += np.kron(self.S[:,task_id].T, np.mat(theta_t).dot(D_t)).T
        L_vectorized = inv(self.A / self.T + self.lam * np.eye(self.d * self.k, self.d * self.k)).dot(self.b) / self.T
        self.L = L_vectorized.reshape((self.k, self.d)).T
        self.revive_dead_components()

    def revive_dead_components(self):
        """ re-initailizes any components that have decayed to 0 """
        for i,val in enumerate(np.sum(self.L, axis = 0)):
            if abs(val) < 10 ** -8:
                self.L[:, i] = np.random.randn(self.d,)

    def predict(self, X, task_id):
        """ Output ELLA's predictions for the specified data on the specified
            task_id.  If using a continuous model (Ridge and LinearRegression)
            the result is the prediction.  If using a classification model
            (LogisticRgerssion) the output is currently a probability.
        """
        if self.base_learner == LinearRegression or self.base_learner == Ridge:
            return X.dot(self.L.dot(self.S[:, task_id]))
        elif self.base_learner == LogisticRegression:
            return 1. / (1.0 + np.exp(-X.dot(self.L.dot(self.S[:, task_id])))) > 0.5

    def predict_probs(self, X, task_id):
        """ Output ELLA's predictions for the specified data on the specified
            task_id.  If using a continuous model (Ridge and LinearRegression)
            the result is the prediction.  If using a classification model
            (LogisticRgerssion) the output is currently a probability.
        """
        if self.base_learner == LinearRegression or self.base_learner == Ridge:
            raise Exception("This base learner does not support predicting probabilities")
        elif self.base_learner == LogisticRegression:
            return np.exp(self.predict_logprobs(X, task_id))

    def predict_logprobs(self, X, task_id):
        """ Output ELLA's predictions for the specified data on the specified
            task_id.  If using a continuous model (Ridge and LinearRegression)
            the result is the prediction.  If using a classification model
            (LogisticRgerssion) the output is currently a probability.
        """
        if self.base_learner == LinearRegression or self.base_learner == Ridge:
            raise Exception("This base learner does not support predicting probabilities")
        elif self.base_learner == LogisticRegression:
            return -logsumexp(np.hstack((np.zeros((X.shape[0], 1)), -X.dot(self.L.dot(self.S[:, task_id])))), axis = 1)

    def score(self, X, y, task_id):
        """ Output the score for ELLA's model on the specified testing data.
            If using a continuous model (Ridge and LinearRegression)
            the score is explained variance.  If using a classification model
            (LogisticRegression) the score is accuracy.
        """
        return self.perf_metric(self.predict(X, task_id), y)

    def get_hessian(self, model, X, y):
        """ ELLA requires that each single task learner provide the Hessian
            of the loss function evaluated around the optimal single task
            parameters.  This funciton implements this for the base learners
            that are currently supported """
        theta_t = model.coef_
        if self.base_learner == LinearRegression:
            return X.T.dot(X)/(2.0 * X.shape[0])
        elif self.base_learner == Ridge:
            return X.T.dot(X)/(2.0 * X.shape[0]) + model.alpha * np.eye(self.d, self.d)
        elif self.base_learner == LogisticRegression:
            preds = 1. / (1.0 + np.exp(-X.dot(theta_t.T)))
            base = np.tile(preds * (1 - preds), (1, X.shape[1]))
            hessian = (np.multiply(X, base)).T.dot(X) / (2.0 * X.shape[0])
            return hessian + np.eye(self.d,self.d) / (2.0 * model.C)

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SparseNet(nn.Module):

    def __init__(self, K:int=32, M:int=128, R_lr:float=0.1, lmda:float=5e-3, device=None):
        super(SparseNet, self).__init__()
        self.K = K
        self.M = M
        self.R_lr = R_lr
        self.lmda = lmda
        # synaptic weights
        self.device = torch.device("cpu") if device is None else device
        self.U = nn.Linear(self.K, self.M, bias=False).to(self.device)
        # responses
        self.R = None
        self.normalize_weights()

    def ista_(self, img_batch):
        # create R
        self.R = torch.zeros((img_batch.shape[0], self.K), requires_grad=True, device=self.device)
        converged = False
        # update R
        optim = torch.optim.SGD([{'params': self.R, "lr": self.R_lr}])
        # train
        while not converged:
            old_R = self.R.clone().detach()
            # pred
            pred = self.U(self.R)
            # loss
            loss = ((img_batch - pred) ** 2).sum()
            loss.backward()
            # update R in place
            optim.step()
            # zero grad
            self.zero_grad()
            # prox
            self.R.data = SparseNet.soft_thresholding_(self.R, self.lmda)
            # convergence
            converged = torch.norm(self.R - old_R) / torch.norm(old_R) < 0.01

    @staticmethod
    def soft_thresholding_(x, alpha):
        with torch.no_grad():
            rtn = F.relu(x - alpha) - F.relu(-x - alpha)
        return rtn.data

    def zero_grad(self):
        self.R.grad.zero_()
        self.U.zero_grad()

    def normalize_weights(self):
        with torch.no_grad():
            self.U.weight.data = F.normalize(self.U.weight.data, dim=0)

    def forward(self, img_batch):
        # first fit
        self.ista_(img_batch)
        # now predict again
        pred = self.U(self.R)
        return pred, self.R




In [103]:
from math import log
import numpy as np
from scipy.special import logsumexp
from scipy.linalg import sqrtm, inv, norm
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression, Lasso
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, explained_variance_score
from sklearn.decomposition import PCA

class ParamShare(object):
    def __init__(self):
        self.net = SparseNet()
        self.base_learner = LogisticRegression
        self.codes = []

    def fit(self, X, y, task_id):
        single_task_model = self.base_learner(max_iter=10000)
        single_task_model.fit(X,y)
        theta_t = torch.tensor(single_task_model.coef_)
        optim = torch.optim.SGD([{'params': self.net.U.weight, "lr": 1e-2}])
        
        running_loss = 0
        code = None
        for c in range(10):
            pred, code = self.net.forward(theta_t)
            loss = ((theta_t - pred) ** 2).sum()
            running_loss += loss.item()
            loss.backward()
            optim.step()
            self.net.zero_grad()
            self.net.normalize_weights()
            #print('Loss', running_loss / (c+1))
        
        self.codes.append(code.clone().detach().numpy()[0])
        tpred = self.predict(X,task_id)
        print()
        print("PMS-BAC:", balanced_accuracy_score(y,tpred))
        print("T-BAC:", balanced_accuracy_score(y,single_task_model.predict(X)))

    def predict(self, X, task_id):
        code = self.codes[task_id]
        theta_t = self.net.U(torch.tensor(code)).detach().numpy()
        sign = 1. / (1.0 + np.exp(-X.dot(theta_t)))
        pred = [1 if p > 0.5 else 0 for p in sign]
        return np.array(pred)


In [104]:
T = 3
d = 128

bacs = [[],[],[]]


for _foldid in range(10):
    print("fold",_foldid)
    ella = ParamShare()

    ### Train
    for task in range(T):
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
        
        pca = PCA(n_components=d)
        pca.fit_transform(X)
        X=pca.transform(X)

        train = foldids[_foldid][1]
        test = foldids[_foldid][2]

        ella.fit(X[train], y[train], task)
 
    ##### Test
    for task in range(T):
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
        
        pca = PCA(n_components=d)
        pca.fit_transform(X)
        X=pca.transform(X)

        train = foldids[_foldid][1]
        test = foldids[_foldid][2]

        pred=ella.predict(X[test], task)
        bac = balanced_accuracy_score(y[test], pred)
        print(task,bac)
        bacs[task].append(bac)

fold 0

PMS-BAC: 0.7798073058942624
T-BAC: 0.8074086508869118

PMS-BAC: 0.8798149437289708
T-BAC: 0.8959276018099547

PMS-BAC: 0.8223209217386738
T-BAC: 0.807354889994566
0 0.7756078375286042
1 0.8640648567119156
2 0.7911586876546357
fold 1

PMS-BAC: 0.78870852402746
T-BAC: 0.8037021902582544

PMS-BAC: 0.8787417333797425
T-BAC: 0.885384035270913

PMS-BAC: 0.8115487670788042
T-BAC: 0.7971320312177952
0 0.7737838346533998
1 0.8692713771899292
2 0.7905394417610565
fold 2

PMS-BAC: 0.7862702906181167
T-BAC: 0.8117376552159161

PMS-BAC: 0.8779005685114283
T-BAC: 0.8781471168348995

PMS-BAC: 0.8244794160247747
T-BAC: 0.8071963108635796
0 0.7854190503432494
1 0.8659067177166724
2 0.792542574500644
fold 3

PMS-BAC: 0.7826587528604119
T-BAC: 0.8102239293886891

PMS-BAC: 0.8813232393549135
T-BAC: 0.899640329504583

PMS-BAC: 0.8122435708150462
T-BAC: 0.7974982354391128
0 0.7603573081833952
1 0.8643839192481726
2 0.7975300542805255
fold 4

PMS-BAC: 0.780463015245624
T-BAC: 0.8139021573804183

PMS-

In [8]:
T = 3
d = 85
k = 32
noise_var = .1

bacs = [[],[],[]]


for _foldid in range(10):
    print("fold",_foldid)
    ella = ELLA(d,k,LogisticRegression, base_learner_kwargs={'max_iter':10000},mu=1,lam=10**-5)

    ### Train
    for task in range(T):
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
        
        pca = PCA(n_components=d)
        pca.fit_transform(X)
        X=pca.transform(X)

        train = foldids[_foldid][1]
        test = foldids[_foldid][2]

        ella.fit(X[train], y[train], task)
 
    ##### Test
    for task in range(T):
        X = DATA["X"][DATA["category"] == task]
        y = DATA["y"][DATA["category"] == task]
        foldids = DATA["folds"][task]
        
        pca = PCA(n_components=d)
        pca.fit_transform(X)
        X=pca.transform(X)

        train = foldids[_foldid][1]
        test = foldids[_foldid][2]

        pred=ella.predict(X[test], task)
        bac = balanced_accuracy_score(y[test], pred)
        print(task,bac)
        bacs[task].append(bac)

fold 0
0 0.7696142530238641
1 0.8519839888618169
2 0.7937192568906208
fold 1
0 0.7611986916334743
1 0.8658051978187724
2 0.7928269709769888
fold 2


/home/rkozik/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.109e+00, tolerance: 1.034e-03
  model = cd_fast.enet_coordinate_descent(


0 0.7606836384439359
1 0.8652830954867154
2 0.7823391641659958
fold 3


/home/rkozik/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.672e+00, tolerance: 8.513e-04
  model = cd_fast.enet_coordinate_descent(


0 0.7542351498873239
1 0.8394100243647755
2 0.7952607061114579
fold 4


...

0 0.7710832788492972
1 0.8514038751595312
2 0.7875865770547354
fold 7


/home/rkozik/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.622e+00, tolerance: 9.290e-04
  model = cd_fast.enet_coordinate_descent(


0 0.7543221369308326
1 0.86267258382643
2 0.7885551825074087
fold 8


/home/rkozik/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.977e-01, tolerance: 1.059e-03
  model = cd_fast.enet_coordinate_descent(


0 0.765324656750572
1 0.8577851258846734
2 0.7870041668033728
fold 9
0 0.7691211256428647
1 0.8583072282167304
2 0.796225311653445


In [106]:
for t in range(T):
    print(t,np.mean(bacs[t]))

0 0.7758572966875313
1 0.8634412344819584
2 0.79371740660875


In [107]:
np.mean(np.array(bacs).flatten()), np.std(np.array(bacs).flatten())

(0.8110053125927466, 0.038171596451482374)